In [1]:
from classes.Dataset import Dataset
from classes.Process_Stages import Process_Stages

ASAP_sas = Dataset(
    df_name="ASAP_sas", 
    model_name="gensim", 
    language="english", 
    process_stages = Process_Stages(
        lower=True, 
        only_text=True, 
        strip_extra_whitespace=True, 
        spelling_check=True, 
        strip_punctuation=True, 
        gensim_remove_stop_words=False, 
        gensim_tokenization=False, 
        gensim_lemmatize=False

    )
)

ASAP_sas.run_all()


Found ASAP_sas with type .csv in (data_saved/basic_processed) and converted it to df
basic processing already done for ASAP_sas
no saving needed because new basic processing done on ASAP_sas


In [2]:
vars(ASAP_sas.process_stages)


{'lower': False,
 'only_text': False,
 'strip_extra_whitespace': False,
 'spelling_check': False,
 'strip_punctuation': False,
 'gensim_remove_stop_words': False,
 'gensim_tokenization': False,
 'gensim_lemmatize': False}